# VRPTW - M3AS

### Definición del problema

In [ ]:
class Cliente:
    def __init__(self, numero, x, y, demanda, inicio, fin, tiempo):
        self.numero = int(float(numero))
        self.x = float(x)
        self.y = float(y)
        self.demanda = float(demanda)
        self.inicio = float(inicio)
        self.fin = float(fin)
        self.tiempo = float(tiempo)

    @property
    def coords(self):
        return np.array([self.x, self.y])

In [ ]:
# Preprocesar los datos de entrada
def procesar_instancia(filename):
    with open(filename, "r") as f:
        lineas = f.readlines()

    # Obtener clientes y capacidad
    clientes = int(lineas[1])
    capacidad = int(lineas[3])

    # Obtener por cada cliente los datos
    lista = []
    for cliente in lineas[5:]:
        lista.append(Cliente(*cliente.strip().split()))

    return {'capacidad': capacidad, 'clientes': lista, 'num': len(lista)}

# Halla la distancia euclidiana entre dos clientes
def hallar_tiempo(i: int, j: int, clientes: [Cliente]):
    return np.linalg.norm(clientes[i].coords - clientes[j].coords)

# Convierte la lista en un pareto set
def obtener_pareto_set(lista):
    objective_values_array = np.vstack([
        [s.num_vehiculos, s.tiempo_total] for s in lista
    ])
    mask = paretoset(objective_values_array, sense=["min", "min"])
    return [solucion for (solucion, m) in zip(lista, mask) if m]

In [ ]:
class Solucion:
    def __init__(self, instancia):
        self.visitados = np.zeros(instancia['num'], dtype=np.bool8)
        self.visitados[0] = True
        self.caminos = [[]]
        self.tiempo_actual = 0
        self.consumo_capacidad = 0
        
        self.capacidad = instancia['capacidad']
        self.clientes = instancia['clientes']

    @property
    def num_vehiculos(self):
        return len(self.caminos)

    @property
    def nodo_actual(self):
        if len(self.caminos[-1]) > 0:
            _ , actual = self.caminos[-1][-1]
        else:
            actual = 0
        return actual

    def camino_en_vehiculo(self, i):
        return self.caminos[i]

    @property
    def completo(self):
        return False not in self.visitados

    @property
    def siguientes_nodos(self):
        i = self.nodo_actual
        no_visitados = np.where(self.visitados == False)[0]
        siguientes_j = []

        for j in no_visitados:
            # Verificar que cumpla la ventana
            tiempo = hallar_tiempo(i, j, self.clientes)
            tiempo_al_llegar = max(self.tiempo_actual + tiempo, self.clientes[j].inicio)
            
            if tiempo_al_llegar + self.clientes[j].tiempo > self.clientes[j].fin:
                continue # No va a poder atenderle antes del cierre
            
            # Verificar que cumpla la capacidad del vehiculo
            if self.consumo_capacidad + self.clientes[j].demanda > self.capacidad:
                continue # Supera la capacidad

            # Se puede visitar
            siguientes_j.append(j)

        return siguientes_j

    def agregar_camino(self, j):
        if j != 0 and self.visitados[j]:
            raise Exception("El cliente ya fue visitado.")
        
        # El sumamos al tiempo lo que cuesta llegar hasta j y su servicio (se incluye la espera si llega antes)
        self.tiempo_actual = max(hallar_tiempo(self.nodo_actual, j, self.clientes) + 
                                 self.tiempo_actual, self.clientes[j].inicio) + self.clientes[j].tiempo
        self.visitados[j] = True
        self.consumo_capacidad = self.consumo_capacidad + self.clientes[j].demanda
        camino = (self.nodo_actual, j)
        self.caminos[-1].append( camino )

    def agregar_vehiculo(self):
        self.caminos.append([])
        self.tiempo_actual = 0
        self.consumo_capacidad = 0

    @property
    def tiempo_total(self):
        """
        Evalua el tiempo recorrido sin tener en cuenta el tiempo esperado
        """
        tiempo = 0.0
        for vehiculo in self.caminos:
            for viaje in vehiculo:
                tiempo = tiempo + hallar_tiempo(*viaje, self.clientes)

        return tiempo

### Implementación del algoritmo M3AS

In [ ]:
# Calcular Función Objetivo delta t
def feromona_a_depositar(solucion: Solucion, norm_vehiculo, norm_tiempo):
    return 1 / (solucion.num_vehiculos / norm_vehiculo + solucion.tiempo_total / norm_tiempo)


def t_max_calculado(feromona: float, p: float):
    return feromona / (1 - p)


def t_min_calculado(t_max: float, m: int):
    return t_max / 2 / m


def feromona_calculada(i, j, tabla_t, tabla_n_2, beta):
    visibilidades = tabla_n_2[i, j] 
    return tabla_t[i,j] * (visibilidades ** beta )


def probabilidades_transicion(i: int, nodos: [int], tabla_t, tabla_n_2, beta: float):
    probabilidades = np.array([ feromona_calculada(i,j, tabla_t, tabla_n_2, beta) for j in nodos ])
    total_probabilidades = probabilidades.sum()
    return probabilidades / total_probabilidades


def seleccionar_siguiente_estado(solucion: Solucion, tabla_t, tabla_n_2, beta: float):
    # Seleccionar siguiente nodo con distribución de probabilidad
    nodos_factibles = solucion.siguientes_nodos

    if len(nodos_factibles) > 0: # Hay nodos
        return np.random.choice(nodos_factibles, p=probabilidades_transicion(
            solucion.nodo_actual, nodos_factibles, tabla_t, tabla_n_2, beta))
    else:
        # Ya es muy tarde o ya atendió su capacidad máxima
        return 0 # Volvemos al deposito


def construir_solucion(instancia: dict, tabla_t, tabla_n_2, beta: float) -> Solucion:
    solucion = Solucion(instancia)
    while not solucion.completo: # While existen estados no visitados
        siguiente = seleccionar_siguiente_estado(solucion, tabla_t, tabla_n_2, beta)
        solucion.agregar_camino(siguiente)

        if siguiente == 0:
            # No hay más nodos siguientes, se debe usar otro auto
            solucion.agregar_vehiculo()

    solucion.agregar_camino(0) # Volver al deposito
    return solucion


def m3as(instancia, m=100, p=0.7, beta=0.7, generaciones=10):

    n = instancia['num']
    tabla_t = np.full((n, n), 1.0) # T_i,j
    tabla_n_2 = np.empty((n, n)) # N_i,j

    # Inicializar visibilidad de tiempo
    for i in range(n):
        for j in range(n):
            tabla_n_2[i,j] = 1.0 / hallar_tiempo(i,j, instancia['clientes']) \
                             if hallar_tiempo(i,j, instancia['clientes']) > 0 \
                             else 1
    
    pareto_set: [Solucion] = []
    pareto_generacion: [Solucion] = []
    
    for _ in trange(generaciones, desc="Generaciones", unit="gen"): # While not condición de parada
        for _ in trange(m, desc="Hormigas", unit="hormiga", leave=False): # Por cada hormiga
            # Construir solución y evaluar
            solucion = construir_solucion(instancia, tabla_t, tabla_n_2, beta)

            # Actaulizar pareto si es que domina
            pareto_generacion.append(solucion)
            pareto_set.append(solucion)
            pareto_set = obtener_pareto_set(pareto_set)

        # Actualizar Feromonas
        tabla_t = tabla_t * (1 - p) # Decadencia

        for Y_Known in pareto_set:
            # Calcular delta t, tmax y tmin
            feromona = feromona_a_depositar(Y_Known, norm_vehiculo=10, norm_tiempo=820.0)
            t_max = t_max_calculado(feromona, p)
            t_min = t_min_calculado(t_max, m)
            # Actualizar
            for ruta in Y_Known.caminos:
                for (i, j) in ruta:
                    tabla_t[i, j] = max(min(tabla_t[i, j] + feromona, t_max), t_min)
    
    return pareto_set, pareto_generacion

In [ ]:
def graficar_m3as(pareto_set, pareto_generacion, save=None):
    fig, ax = plt.subplots(figsize=(10, 8))

    ax.plot([s.tiempo_total for s in pareto_generacion],
            [s.num_vehiculos for s in pareto_generacion],
            'bo',
            label="Soluciones")

    ax.plot([s.tiempo_total for s in pareto_set],
            [s.num_vehiculos for s in pareto_set],
            'ro',
            label="Pareto Set")

    ax.set(xlabel="Tiempo de viaje",
           ylabel="Nº de Vehículos",
           title="Frente Pareto (M3AS)")
    ax.yaxis.set_major_locator(ticker.MultipleLocator())
    ax.legend(loc='upper right')

    if save:
        os.makedirs("figs", exist_ok=True)
        plt.savefig(f"figs/{save}")